<a href="https://colab.research.google.com/github/dh610/ai-intensive2/blob/main/lab2_LM_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/ai-intensive2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/ai-intensive2'
/content/drive/MyDrive/ai-intensive2


# Lab 2 : Language Modeling

@copyright:
    (c) 2023. iKnow Lab. Ajou Univ., All rights reserved.

M.S. Student: Wansik-Jo (jws5327@ajou.ac.kr)

# For assignment

- Python code의 주석 처리되어있는 부분을 구현하면 됩니다.
- MD 형식의 Cell의 [BLANK] 부분을 채우면 됩니다.
- MD 형식의 Cell의 [ANSWER] 부분 이후에 답을 작성하면 됩니다.
- 조교에게 퀴즈의 답과 함께 코드 실행 결과를 보여준 뒤, BB에 제출 후 가시면 됩니다.


# 목차
1. [Language Modeling](##1.-Language-Modeling)
2. [N-gram Language Model](##2.-N-gram-Language-Model)
3. [Perplexity](##3.-Perplexity)
4. [Generalization](##4.-Generalization)
---

## 1. Language Modeling



### 1.1. Language Modeling

- 언어 모델링(Language Modeling)은 주어진 단어들의 나열에 대해 그 확률을 예측하는 작업이다.

- 언어 모델은 다음과 같은 다양한 분야에서 활용된다.

  - 기계 번역(Machine Translation)
  - 오타 교정(Spell Correction)
  - 음성 인식(Speech Recognition)
  - 문장 자동 완성(Sentence Completion)
  - 문장 유사도(Sentence Similarity)
  - 감성 분석(Sentiment Analysis)
  - 질의 응답(Question Answering)
  - 챗봇(Chatbot)

- 언어 모델링은 다음과 같은 다양한 방법으로 수행된다.

    - N-gram Language Model
    - Neural Network Language Model
    - Transformer Language Model

### 1.2 Markov Assumption

- 언어 모델링은 다음과 같은 Markov Assumption을 기반으로 한다.

    - 어떤 단어의 확률은 그 이전 단어들에만 의존한다.


- 예를 들어, 다음 확률을 계산하고자 한다고 하자.

$$ P(w_1, w_2, \cdots, w_n) $$

- 조건부 확률에 의해,

$$ \Pi_{i=1}^{n} P(w_i | w_1, w_2, \cdots, w_{i-1}) $$

- Chain Rule에 의해,

$$ P(w_1, w_2, \cdots, w_n) = P(w_1) P(w_2 | w_1) P(w_3 | w_1, w_2) \cdots P(w_n | w_1, w_2, \cdots, w_{n-1}) $$

! 이는 모든 단어의 확률을 계산하기 위해 모든 이전 단어들을 고려해야 한다는 것을 의미한다. -> 비현실적

- 따라서, Markov Assumption을 적용하면,

$$ P(w_1, w_2, \cdots, w_n) = \Pi_{i=1}^{n} P(w_i | w_1, w_2, \cdots, w_{i-1}) \approx \Pi_{i=1}^{n} P(w_i | w_{i-1}) $$



## 2. N-gram Language Model


In [ ]:
def sentence_to_ngrams(sentence, n):
    """
    Converts a sentence into a list of n-grams.

    Parameters
    ----------
    sentence : str
        The sentence to convert.
    n : int
        The length of the n-grams.

    Returns
    -------
    list
        The list of n-grams.
    """
    # YOUR CODE HERE
    sentence = "<s> " + sentence + " </s>"
    sentence = sentence.lower()
    sentence = sentence.split()

    ngrams = []
    for i in range(len(sentence)-n+1):
        ngrams.append(sentence[i:i+n])
    return ngrams

sentence_to_ngrams("I do not like green eggs and ham", 2)

[['<s>', 'i'],
 ['i', 'do'],
 ['do', 'not'],
 ['not', 'like'],
 ['like', 'green'],
 ['green', 'eggs'],
 ['eggs', 'and'],
 ['and', 'ham'],
 ['ham', '</s>']]

In [ ]:
# n-gram LM

def ngram_LM(n, train_data):
    """
    Builds an n-gram language model.

    Parameters
    ----------
    n : int
        The length of the n-grams.
    train_data : list
        The training data.

    Returns
    -------
    dict
        The n-gram language model.
    """
    # YOUR CODE HERE
    ngram_lm = {}
    for sentence in train_data:
        ngrams = sentence_to_ngrams(sentence, n)
        for ngram in ngrams:
            ngram = tuple(ngram)
            if ngram not in ngram_lm:
                ngram_lm[ngram] = 1
            else:
                ngram_lm[ngram] += 1
    return ngram_lm

ngram_LM(3, ["I do not like green eggs and ham", "I do not like them Sam I am"])


{('<s>', 'i', 'do'): 2,
 ('i', 'do', 'not'): 2,
 ('do', 'not', 'like'): 2,
 ('not', 'like', 'green'): 1,
 ('like', 'green', 'eggs'): 1,
 ('green', 'eggs', 'and'): 1,
 ('eggs', 'and', 'ham'): 1,
 ('and', 'ham', '</s>'): 1,
 ('not', 'like', 'them'): 1,
 ('like', 'them', 'sam'): 1,
 ('them', 'sam', 'i'): 1,
 ('sam', 'i', 'am'): 1,
 ('i', 'am', '</s>'): 1}

In [ ]:
def ngram_prob(ngram, ngram_lm):
    """
    Calculates the probability of an n-gram.

    Parameters
    ----------
    ngram : tuple
        The n-gram.
    ngram_lm : dict
        The n-gram language model.

    Returns
    -------
    float
        The probability of the n-gram.
    """
    # YOUR CODE HERE
    ngram = tuple(ngram)
    ngram_prob = ngram_lm[ngram] / sum(ngram_lm.values())
    return ngram_prob

ngram_prob(("i", "do", "not"), ngram_LM(3, ["I do not like green eggs and ham", "I do not like them Sam I am"]))

0.125

## 3. Perplexity

$$ PP(W) = P(w_1, w_2, \cdots, w_n)^{-\frac{1}{n}} = \sqrt[n]{\frac{1}{P(w_1, w_2, \cdots, w_n)}} $$

In [ ]:
#perplexity

def perplexity(sentence, n, ngram_lm):
    """
    Calculates the perplexity of a sentence.

    Parameters
    ----------
    sentence : str
        The sentence.
    n : int
        The length of the n-grams.
    ngram_lm : dict
        The n-gram language model.

    Returns
    -------
    float
        The perplexity of the sentence.
    """
    # YOUR CODE HERE
    ngrams = sentence_to_ngrams(sentence, n)
    perplexity = 1
    for ngram in ngrams:
        perplexity *= 1/ngram_prob(ngram, ngram_lm)
    perplexity = perplexity**(1/len(ngrams))
    return perplexity

perplexity("I do not like green eggs and ham", 3, ngram_LM(3, ["I do not like green eggs and ham", "I do not like them Sam I am"]))

12.337686603263526

## 4. Generalization

!What is the problem of N-gram Language Model?

[ANSWER] :

Laplace Smoothing

$$ P_{Laplace}(w_i | w_{i-1}) = \frac{C(w_{i-1}, w_i) + 1}{C(w_{i-1}) + V} $$